In [1]:
import ee

In [1]:
def generate_collection(plot):
    
    # Extract geometry and plot id
    plot = ee.Feature(plot)
    roi = plot.geometry()
    plot_id = plot.get('codigo_con')
    
    # Select the first image that covers the plot, the timerange, and both polarizations
    orbit_det = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD')
                         .filterBounds(roi)
                         .filterDate(ee.Date('2017-01-01'), ee.Date('2022-12-31'))
                         .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                         .first())
 
    # Extract node
    orbit_node = orbit_det.get('orbitProperties_pass')
    # Extract relative orbit number
    orbit_num = orbit_det.get('relativeOrbitNumber_start')
    
    # Build relevant Image Collection
    s1_coll = (ee.ImageCollection('COPERNICUS/S1_GRD')
               .filterBounds(roi)
               .filterDate(ee.Date('2017-01-01'), ee.Date('2022-12-31'))
               .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
               .filter(ee.Filter.eq('orbitProperties_pass', orbit_node))
               .filter(ee.Filter.eq('relativeOrbitNumber_start', orbit_num))
               .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
               .sort('date')
               .map(lambda img: img.clip(roi)))
    
    # function to create a list of yearly image collections
    def list_of_yearly_coll(year):
        return s1_coll.filterDate(start_list.get(year), end_list.get(year))
    
    list_yr_coll = year_list.map(list_of_yearly_coll)
 
    # function to extract the first image of every month from a yearly image collection
    def monthly_ts(year):
        yearly_coll = ee.ImageCollection(list_yr_coll.get(year))
        return month_list.map(lambda m: yearly_coll.filter(ee.Filter.calendarRange(m,m,'month')).first())
    
    # Create monthly time series over all six years
    ts = year_list.map(monthly_ts).flatten()
    
    # Create image collection
    ts_coll = ee.ImageCollection.fromImages(ts)
    
    # Create list of images to analyze (back and forth conversion eliminates NAs)
    ts_list = ts_coll.toList(ts_coll.size()) 
    
    # Generate timestamps
    timestamplist = (ee.List((ts_coll.aggregate_array('date').map(lambda d: ee.String('T').cat(ee.String(d)))))
                     .splice(start = 0, count = 1))
    
    # Run algorithm at 5% significance
    result = ee.Dictionary(change_maps(ts_list, median=True, alpha=0.05))
    
    # Extract the bmap image, add plot id and rename bands
    bmap = ee.Image(result.get('bmap')).set('plot_id', plot_id).rename(timestamplist)
    
    # Export the generated image
    assetexport = ee.batch.Export.image.toAsset(bmap,
                                                description='assetExportTask',
                                                pyramidingPolicy={".default":'sample'},
                                                assetId='users/peterkamal/change_detection', 
                                                scale=10, maxPixels=1e9)
    
    return assetexport.start()